In [73]:

import csv
import os
import timeit

import argparse
import h5py
import logging
import numpy as np
import ufl

from dolfinx import cpp, fem, io, mesh
from mpi4py import MPI
from petsc4py import PETSc

import commons, configs, constants

markers = commons.SurfaceMarkers()

In [74]:
data_dir = "mesh/study_2/test6/470-470-5_000-000-000/"
tetpath = os.path.join(data_dir, "tetr.h5")
tripath = os.path.join(data_dir, "tria.h5")

In [75]:
geo2d = h5py.File(tripath, "r")
geo3d = h5py.File(tetpath, "r")

In [76]:
geo2d.keys(), geo3d.keys()

(<KeysViewHDF5 ['data0', 'data1', 'data2']>,
 <KeysViewHDF5 ['data0', 'data1', 'data2']>)

In [77]:
geo2d['data0'].shape, geo2d['data1'].shape, geo2d['data2'].shape

((420670, 3), (547158, 3), (547158,))

In [78]:
geo3d['data0'].shape, geo3d['data1'].shape, geo3d['data2'].shape

((420670, 3), (1738202, 4), (1738202,))

In [79]:
current3d = h5py.File("mesh/study_2/test6/470-470-5_000-000-000/current.h5", "r")

In [80]:
values = current3d['Function']['f']['0']

In [90]:
comm = MPI.COMM_WORLD
with io.XDMFFile(comm, tetpath.replace(".h5", ".xdmf"), "r") as infile3:
    domain = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim - 1)
with io.XDMFFile(comm, tripath.replace(".h5", ".xdmf"), "r") as infile2:
    domain2 = infile2.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
    ft = infile2.read_meshtags(domain, name="Grid")
meshtags = mesh.meshtags(domain, 2, ft.indices, ft.values)

In [91]:
W = fem.FunctionSpace(domain, ("Lagrange", 1))
current = fem.Function(W)
cdf_fun = fem.Function(W)
cdf_fun2 = fem.Function(W)

In [93]:
ds = ufl.Measure('ds', domain=domain, subdomain_data=meshtags)
area_left_cc = fem.assemble_scalar(fem.form(1 * ds(markers.left_cc)))
area_right_cc = fem.assemble_scalar(fem.form(1 * ds(markers.right_cc)))

In [94]:
valz = set(np.asarray(geo2d['data1']).flatten())
for dof in valz:
    current.vector.setValueLocal(dof, values[dof][2])

In [ ]:
min_cd = np.min(current.x.array)
max_cd = np.max(current.x.array)
cd_space = np.linspace(min_cd, max_cd, num=1000)
cdf_values = []
Id = ufl.Identity(3)
EPS = 1e-30


def check_condition(v1, check_value=1):
    v2 = lambda x: check_value * (x[0] + EPS) / (x[0] + EPS)
    cdf_fun.interpolate(v2)
    return ufl.conditional(ufl.le(v1, cdf_fun), 1, 0)

for v in cd_space:
    new_v = fem.Expression(check_condition(current, v), W.element.interpolation_points())
    cdf_fun2.interpolate(new_v)
    lpvalue = fem.assemble_scalar(fem.form(cdf_fun2 * ds(markers.left_cc))) / area_left_cc
    rpvalue = fem.assemble_scalar(fem.form(cdf_fun2 * ds(markers.right_cc))) / area_right_cc
    cdf_values.append({'i [A/m2]': v, "p_left": lpvalue, "p_right": rpvalue})
stats_path = os.path.join(data_dir, 'cdf.csv')
with open(stats_path, 'w') as fp:
    writer = csv.DictWriter(fp, fieldnames=['i [A/m2]', 'p_left', 'p_right'])
    writer.writeheader()
    for row in cdf_values:
        writer.writerow(row)